In [ ]:
from scipy.special import rel_entr
from utils import *

In [ ]:
def kl_div(x, y):
    fw = np.mean([np.sum(rel_entr(x[:512], y[:512])), np.sum(rel_entr(y[:512], x[:512]))])
    bw = np.mean([np.sum(rel_entr(x[512:], y[512:])), np.sum(rel_entr(y[512:], x[512:]))])
    return np.mean([fw, bw])


In [ ]:
df_lbp_tr = pd.read_csv('df_lbp_train', index_col=["record", "augm_idx"])
df_lbp_te = pd.read_csv('df_lbp_test', index_col=0)

In [ ]:
x = df_lbp_tr#.iloc[:100]
y = x

def kl_wrap(t):
    return (*t, kl_div(x.iloc[t[0]], y.iloc[t[1]]))
ij = [(i,j) for i in range(x.shape[0]) for j in range(i)]

with closing(Pool(processes=NUM_PROCESSES)) as pool:
    print(f'pid: {os.getpid()}\n', end='')
    r = list(tqdm(pool.map(kl_wrap, ij), total=len(ij)))

gram_tr = np.zeros((x.shape[0],y.shape[0]))
for i,j,d in tqdm(r):
    gram_tr[i,j] = gram_tr[j,i] = d
df_gram_tr = pd.DataFrame(gram_tr, index=x.index, columns=y.index)
df_gram_tr.to_csv("df_gram_tr.csv.gz", compression='gzip')
# df = pd.read_csv('df_gram_tr.csv.gz', index_col=[0,1], header=[0,1])

In [ ]:
x = df_lbp_te#.iloc[:5]
y = df_lbp_tr#.iloc[:20]

def kl_wrap(i):
    return (i, [kl_div(x.iloc[i], y.iloc[j]) for j in range(y.shape[0])])
with closing(Pool(processes=NUM_PROCESSES)) as pool:
    print(f'pid: {os.getpid()}\n', end='')
    r = list(tqdm(pool.imap(kl_wrap, range(x.shape[0])), total=x.shape[0]))
gram = [x[1] for x in sorted(r)]
df_gram = pd.DataFrame(gram, index=x.index, columns=y.index)
df_gram.to_csv("df_gram_te.csv.gz", compression='gzip')
# df = pd.read_csv('df_gram_te.csv.gz', index_col=0, header=[0,1])

In [ ]:
with closing(Pool(processes=NUM_PROCESSES, maxtasksperchild=1)) as pool:
    print(f'pid: {os.getpid()}\n', end='')
    kxys = list(tqdm(pool.imap(readcgw1_wrap, records), total=len(records)))
